<a href="https://colab.research.google.com/github/prubinstreit/Cryo_Scraper/blob/main/Working_Fairfax_Cryo_Bank_Scraper_Each_Profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Running this notebook on Google Colab? Then run this cell! Otherwise, please use the cell above!

!pip install selenium
!pip install helium
!apt-get update 
!apt install chromium-chromedriver
!pip install requests


     |████████████████████████████████| 963 kB 5.0 MB/s 
     |████████████████████████████████| 138 kB 55.0 MB/s 
     |████████████████████████████████| 359 kB 65.4 MB/s 
     |████████████████████████████████| 3.6 MB 46.4 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 58 kB 5.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 26.1 MB 69.2 MB/s 
     |████████████████████████████████| 904 kB 48.7 MB/

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from datetime import date
import re
import time

import pandas as pd
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
from helium import *
import requests

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


In [ ]:
url = 'https://fairfaxcryobank.com/search/searchresults.aspx'
driver.get(url)
element = driver.find_element(By.CLASS_NAME, "submit")
element.click()

In [ ]:
time.sleep(1)
results_html = BeautifulSoup(driver.page_source, 'html.parser')
h = results_html.find_all('div', attrs={'class': 'total'})[0]
pages = h.get_text()[-2:]
pages = int(pages)
pages

36

In [ ]:
def next_page():
  try:
    #driver.switch_to().accept()
    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='content']/div/div[3]/div/div[1]/div[4]/div[2]/a"))).click()
    #driver.implicitly_wait(10)
    #driver.switch_to().alarm().accept()
  except:
    pass

In [ ]:
def profiles():
  soup = BeautifulSoup(driver.page_source, 'html.parser')
  el = soup.find_all(class_='left', href=True)
  blocks = len(el)
  url_profile = []
  for i in range(blocks):
    url_profile.append(el[i]['href'])
  df1=pd.DataFrame({'ID':url_profile, 'site':"https://fairfaxcryobank.com/search/"})

  list = [soup, df1]
  del list
  df1['url']=df1.site.str.cat(df1.ID)
  return df1

#profiles()

In [ ]:
#get all profile url's from search page
df_list=[]
for i in range(1):
  df_final = profiles()
  df_final = pd.DataFrame(df_final['url'])
  df_list = df_final
  del df_final

for j in range(1,pages):
  next_page()
  time.sleep(1)
  df_final = profiles()
  df_final = pd.DataFrame(df_final['url'])
  df_list = pd.concat([df_list,df_final])
  del df_final 
#now = datetime.now()
#df_list['Date']= now
df_list
user_num = len(df_list)

In [ ]:
def page_data(str):
  driver.get(str)
  soup = BeautifulSoup(driver.page_source,'html.parser')
  #descriptives
  data1 = soup.find('ul', attrs={'class': 'list-info'})
  new = [x.get_text() for x in data1.find_all('li')]
  data1 = {"Ancestry": new[0][9:],
          "Ethnic Background": new[1][18:],
          "Degree":new[2][7:],
          "Occupation":new[3][11:],
          "Interests":new[4][10:],
          "Photo Matching":new[6][15:],
          "CMV Status":new[7][11:],
          "Age Range at Donation":new[8][22:],
          "Astrological Sign":new[9][18:],
          "Favorite Subject":new[10][17:],
          "Religion":new[11][9:],
          "Favorite Pet":new[12][13:],
          "Personal Goals":new[13][15:],
          "Talents/Hobbies":new[14][16:],                          
          }
  df_extra = pd.DataFrame(data1,index=[0])

  #Vials and Price Availabilty
  bl01 = [x.get_text() for x in soup.find_all('tr', attrs={'class': 'sys-product-cnt'})]
  arr = [a.split("\n") for a in bl01]

  if len(arr)==0:
   data = {"IUI": '',
          "IUI ART": '',
          "ICI":'',
          "ICI ART":'',
          "IVF":''
         }
  if len(arr)==1:
   data = {"IUI": ''.join(arr[0]),
          "IUI ART": '',
          "ICI":'',
          "ICI ART":'',
          "IVF":''
         }
  elif len(arr)==2:
   data = {"IUI": ''.join(arr[0]),
          "IUI ART": ''.join(arr[1]),
          "ICI":'',
          "ICI ART":'',
          "IVF":''
         }
  elif len(arr)==3:
   data = {"IUI": ''.join(arr[0]),
          "IUI ART": ''.join(arr[1]),
          "ICI":''.join(arr[2]),
          "ICI ART":'',
          "IVF":''
        }
  elif len(arr)==4:
   data = {"IUI": ''.join(arr[0]),
          "IUI ART": ''.join(arr[1]),
          "ICI":''.join(arr[2]),
          "ICI ART":''.join(arr[3]),
          "IVF":''
        }
  elif len(arr)==5:
   data = {"IUI": ''.join(arr[0]),
          "IUI ART": ''.join(arr[1]),
          "ICI":''.join(arr[2]),
          "ICI ART":''.join(arr[3]),
          "IVF":''.join(arr[4])
        }
  df = pd.DataFrame(data,columns = ['IUI', 'IUI ART','ICI','ICI ART','IVF'], index=[0])

  #height weight and hair
  bl01 = [x.get_text() for x in soup.find_all('div', attrs={'class': 'info'})]
  arr = [a.split("\n") for a in bl01]
  hair = arr[4][2]+ arr[4][1]
  hair = hair.split('\t')
  hair = hair[16]+hair[8]
  #print(hair)
  data = {'Height':  arr[0],
          'Weight':  arr[1],
          'Eye Color':  arr[2],
          'Skin Tone':  arr[3],
          'Hair':  hair,
          'Blood Type':  arr[5],
      }
  df1 = pd.DataFrame(data)

  #expanded genetic testing CONTINUES AFTER IF STATEMENT
  data = soup.find('div', attrs={'class': 'row rr-01'}).get_text()
  #print("row rr is this long:", len(data))
  df4 = pd.DataFrame(data,columns = ['Expanded Genetic Testing'], index=[0])
  
  #about the donor
  if len(data) <=4:
    data = soup.find('div', attrs={'class': 'text'}).get_text()
    data2 = {"About": " ",              
            }
    #print("here also")
    df2 = pd.DataFrame(data2,columns = ['About'], index=[0])
  
  elif len(data) >5 :
    data_except = soup.find_all('div', attrs={'class': 'text'})[1].get_text()
    #print(data_except)
    data2 = {"About": data_except,              
            }
    df2 = pd.DataFrame(data2,columns = ['About'], index=[0])
    #print(df2)


  #df4 = pd.DataFrame(data,columns = ['Expanded Genetic Testing'], index=[0])


  #donor category
  data = soup.find_all('div', attrs={'class': 'ts-02'})[0]
  data3 = {"Donor Category": data.get_text(),              
        }
  df3 = pd.DataFrame(data3,columns = ['Donor Category'], index=[0])


  

  df_final= pd.merge(df,df1,left_index=True, right_index=True)
  df_final= pd.merge(df_final,df_extra,left_index=True, right_index=True)
  df_final= pd.merge(df_final,df2,left_index=True, right_index=True)
  df_final= pd.merge(df_final,df3,left_index=True, right_index=True)
  df_final= pd.merge(df_final,df4,left_index=True, right_index=True)

  df_final['ID']=str[-15:]

  return df_final

In [ ]:
#driver.get(df_list.iloc[100,0])
#
#soup = BeautifulSoup(driver.page_source,'html.parser')
##about the donor
#data = soup.find_all('div', attrs={'class': 'text'})[1].get_text()
#data
#data2 = {"About": data,              
#        }

#data = soup.find('div', attrs={'class': 'row rr-01'}).get_text()
#print(len(data))
#data4 = {"Expanded Genetic Testing": data,              
#        }
#df4 = pd.DataFrame(data4,columns = ['Expanded Genetic Testing'], index=[0])
#page_data(df_list.iloc[250,0])


In [ ]:
df_list.iloc[100,0]

'https://fairfaxcryobank.com/search/donorprofile.aspx?number=5069&s=1'

In [ ]:
page_data(df_list.iloc[100,0])

,IUI,IUI ART,ICI,ICI ART,IVF,Height,Weight,Eye Color,Skin Tone,Hair,Blood Type,Ancestry,Ethnic Background,Degree,Occupation,Interests,Photo Matching,CMV Status,Age Range at Donation,Astrological Sign,Favorite Subject,Religion,Favorite Pet,Personal Goals,Talents/Hobbies,About,Donor Category,Expanded Genetic Testing,ID
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"5'11""(180 cm)",210 lb(95 kg),Hazel,Light,Brown/Wavy,B+,Caucasian,German-Irish-Native American/Irish,MS/Geoinfmtcs & Geospatl Intel,Geospatial Scientist,"Edible landscapes, Hiking, Mycology",Yes,Positive,35-39,Libra,"History, Mathematics, Natural Science, Technology",Christian (Catholic),Dog,"Financial Security, Further Education, Marriag...","French Horn, Wood Carving",,ID Graduate,\n\n\n,number=5069&s=1


In [ ]:
#aggregate each user data (will take a long time)
df_list_final=[]

df_final = page_data(df_list.iloc[0,0])
df_list_final = df_final
#del df_final

#print(df_list_final)
df_list_final = df_list_final.drop(range(1))
for j in range(user_num):
  #driver.get(df_list.iloc[j,0])
  #time.sleep(1)
  print("j is:",j,"url is",df_list.iloc[j,0])
  df_final = page_data(df_list.iloc[j,0])
  df_list_final = pd.concat([df_list_final,df_final])

  #print(df_list_final)
  del df_final
 
now = datetime.now()
df_list_final['Date']= now
#df_list_final['ID'] = df_list['url']
df_list_final

#print(df_list.head(10))
#print("df_list size:",len(df_list))

j is: 0 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number=6281&s=1
j is: 1 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number=6087&s=1
j is: 2 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number=5866&s=1
j is: 3 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number=5852&s=1
j is: 4 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number=5834&s=1
j is: 5 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number=6424&s=1
j is: 6 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number=6331&s=1
j is: 7 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number=6426&s=1
j is: 8 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number=5619&s=1
j is: 9 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number=6414&s=1
j is: 10 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number=5595&s=1
j is: 11 url is https://fairfaxcryobank.com/search/donorprofile.aspx?number

,IUI,IUI ART,ICI,ICI ART,IVF,Height,Weight,Eye Color,Skin Tone,Hair,Blood Type,Ancestry,Ethnic Background,Degree,Occupation,Interests,Photo Matching,CMV Status,Age Range at Donation,Astrological Sign,Favorite Subject,Religion,Favorite Pet,Personal Goals,Talents/Hobbies,About,Donor Category,Expanded Genetic Testing,ID,Date
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"5'8""(172 cm)",172 lb(78 kg),Brown,Medium,Black/Straight,A+,Asian,Nepalese/Nepalese,PhD/Analytical Chemistry,Postdoctoral fellow,"Family Activities, Indiviaual Sports, Team Sports",Yes,Positive,29-34,Scorpio,Natural Science,Hindu,Dog,"Further Education, Rewarding Career, To Be Hap...","Badminton, Cricket, Tennis, Volleyball",\nGet to know the donor\nDonor 6281 is a highl...,ID Graduate,\n\n\n\nThis donor has been tested for an expa...,number=6281&s=1,2022-02-28 21:05:23.036104
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"6'0""(182 cm)",213 lb(96 kg),Brown,Medium Light,Black/Straight,O+,Multi,Mexican/English-Mexican,MS/Engineering Management,Engineer,"Baseball, Cooking, Guitar, Piano, Singing",Yes,Positive,29-34,Taurus,Mathematics,Christian,Dog,"Further Education, Rewarding Career, Success","Baseball, Cooking, Piano, Singing musicals, So...",\nGet to know the donor\nDonor 6087 is a sweet...,Non-ID Graduate,\n\n\n\nThis donor has been tested for an expa...,number=6087&s=1,2022-02-28 21:05:23.036104
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"5'9""(175 cm)",171 lb(77 kg),Brown,Light,Black/Straight,O+,Asian,Vietnamese/Vietnamese,PhD/Material Sci & Engineering,Postdoctoral researcher,"Comics, Individual Sports, Photography, Readin...",Yes,Positive,29-34,Scorpio,"History, Languages, Mathematics, Natural Scien...",Other (Atheist),Other (Turtle),"Financial Security, Further Education, Improve...","Adventure novels, Badminton, Japanase manga, L...",\nGet to know the donor\nDonor 5866 is a man w...,ID Graduate,\n\n\n\nThis donor has been tested for an expa...,number=5866&s=1,2022-02-28 21:05:23.036104
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"5'10""(177 cm)",163 lb(73 kg),Brown,Light,Black/Straight,B+,Asian,Chinese/Chinese,MS/Math and Computational Fin,Data Scientist/Programmer Anal,"Basketball, Running, Tennis",Yes,Positive,35-39,Virgo,"Mathematics, Technology",Christian,Dog,"Further Education, Success, To Help People","Basketball, Tennis",\nGet to know the donor\nDonor 5852 is a very ...,Non-ID Graduate,\n\n\n\nThis donor has been tested for an expa...,number=5852&s=1,2022-02-28 21:05:23.036104
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"6'0""(182 cm)",266 lb(120 kg),Brown,Medium,Black/Wavy,O+,Latino,Mexican/Mexican,BA/Psyc & Spanish,Student,"Farming, Gardening, Helping environment, Outdo...",Yes,Positive,18-22,Virgo,"Languages, Social Science",Christian (Catholic),Other (Dogs and Cats),Financial Security,"Experiencing new cultures, Riding motocycle, W...","\nGet to know the donor\nDonor 5834 is calm, s...",ID,\n\n\n\nThis donor has been tested for

In [ ]:
dftoday = df_list_final.copy(deep=True)
today = datetime.now()
date1 = today.strftime("%b-%d")
from google.colab import files
dftoday.to_csv('dftoday %s.csv'%date1) 
files.download('dftoday %s.csv'%date1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

#df_list_final.replace(regex=['number='], value='', inplace=True)
#df_list_final.replace(regex=['&s=1'], value='',inplace = True)
#df_list_final.replace(regex=['er='], value='',inplace = True)
#df_list_final.replace(regex=['Intrauterine Insemination Ready / Washed/'], value='', inplace = True)
#df_list_final.replace(regex=['IUI 10 million total motile cells'], value='', inplace = True) #IUI
#df_list_final.replace(regex=['Intrauterine Insemination Ready individually or combined/ Washed/ >6 million total motile cells. Vials are only sold in pairs. Fee is for 2 vials. '], value='', inplace = True) #IUI ART
#df_list_final.replace(regex=['Intracervical Insemination Ready / Unwashed/ 10 million total motile cells'], value=' ', inplace = True) #ICI 
#df_list_final.replace(regex=['Intracervical Insemination Ready individually or combined/ Unwashed/ >6 million total motile cells. Vials are only sold in pairs. Fee is for 2 vials.'], value=' ', inplace = True) #ICI ART
#df_list_final.replace(regex=['In Vitro Fertilization Unit / Unwashed/ 5 million total motile cells'], value=' ', inplace = True) #IVF
#df_list_final.replace(regex=['\n\n\n\n'], value='', inplace = True)
#df_list_final.replace(regex=['\nGet to know the donor\n'], value='', inplace = True)
#df_list_final.replace(regex=['Add to Cart'], value='', inplace = True)
#df_list_final.replace(regex=['12345678910'], value=' Max10 ', inplace = True)
#df_list_final.replace(regex=['123456789'], value=' Max9 ', inplace = True)
#df_list_final.replace(regex=['12345678'], value=' Max8 ', inplace = True)
#df_list_final.replace(regex=['1234567'], value=' Max7 ', inplace = True)
#df_list_final.replace(regex=['123456'], value=' Max6 x', inplace = True)
#df_list_final.replace(regex=['12345'], value=' Max5 ', inplace = True)
#df_list_final.replace(regex=['1234'], value=' Max4 ', inplace = True)
#df_list_final.replace(regex=['246810'], value=' EvenMax10 ', inplace = True)
#df_list_final
#

In [ ]:
df = df_list_final
df['All'] = df['IUI'].astype(str) +"@"+ df['IUI ART'].astype(str) +"@"+ df['ICI'].astype(str) +"@"+ df['ICI ART'].astype(str) +"@"+ df['IVF'].astype(str)+"@"
df['All']=df['All'].astype(str) 
df['IUI ART'] = df['All'].str.extract(pat='(IUI ART[/$0-9,A-Za-z .>]*)')
df['IUI1'] = df['All'].str.extract(pat='(IUII[/$0-9,A-Za-z .>]*)')
df['ICI'] = df['All'].str.extract(pat='(ICII[/$0-9,A-Za-z .>]*)')
df['ICI ART'] = df['All'].str.extract(pat='(ICI ART[/$0-9,A-Za-z .>]*)')
df['IVF'] = df['All'].str.extract(pat='(IVF[/$0-9,A-Za-z .>]*)')


df['IUI Number']= df['IUI1']
df['IUI ART Number']= df['IUI ART']
df['ICI Number']= df['ICI']
df['ICI ART Number']= df['ICI ART']
df['IVF Number']= df['IVF']

df= df.replace({'IUI Number': { 'IUIInt[/$0-9,A-Za-z .>]*12345678910112345678910Add to Cart$': ' Max10 ', 
                                'IUIInt[/$0-9,A-Za-z .>]*1234567891123456789Add to Cart$': ' Max9 ',
                                'IUIInt[/$0-9,A-Za-z .>]*12345678112345678Add to Cart$': ' Max8 ',
                                'IUIInt[/$0-9,A-Za-z .>]*123456711234567Add to Cart$': ' Max7 ',
                                'IUIInt[/$0-9,A-Za-z .>]*1234561123456Add to Cart$': ' Max6 ',
                                'IUIInt[/$0-9,A-Za-z .>]*12345112345Add to Cart$': ' Max5 ',
                                'IUIInt[/$0-9,A-Za-z .>]*Please Call$': ' Please Call ',   
                                'IUIInt[/$0-9,A-Za-z .>]*Notify Me$': ' Notify Me ',                                                                        
                                }}, regex=True)

df= df.replace({'IUI ART Number': { 'IUI ART[/$0-9,A-Za-z .>]*2468102246810Add to Cart$': ' EvenMax10 ', 
                                    'IUI ART[/$0-9,A-Za-z .>]*246822468Add to Cart$': ' EvenMax8 ',
                                    'IUI ART[/$0-9,A-Za-z .>]*2462246Add to Cart$': ' EvenMax6 ',
                                    'IUI ART[/$0-9,A-Za-z .>]*24224Add to Cart$': ' EvenMax4 ',
                                    'IUI ART[/$0-9,A-Za-z .>]*Please Call$': ' Please Call ',   
                                    'IUI ART[/$0-9,A-Za-z .>]*Notify Me$': ' Notify Me ',                                                                        
                                }}, regex=True)

df= df.replace({'ICI Number': { 'ICIInt[/$0-9,A-Za-z .>]*12345678910112345678910Add to Cart$': ' Max10 ', 
                                'ICIInt[/$0-9,A-Za-z .>]*1234567891123456789Add to Cart$': ' Max9 ',
                                'ICIInt[/$0-9,A-Za-z .>]*12345678112345678Add to Cart$': ' Max8 ',
                                'ICIInt[/$0-9,A-Za-z .>]*123456711234567Add to Cart$': ' Max7 ',
                                'ICIInt[/$0-9,A-Za-z .>]*1234561123456Add to Cart$': ' Max6 ',
                                'ICIInt[/$0-9,A-Za-z .>]*12345112345Add to Cart$': ' Max5 ',
                                'ICIInt[/$0-9,A-Za-z .>]*Please Call$': ' Please Call ',   
                                'ICIInt[/$0-9,A-Za-z .>]*Notify Me$': ' Notify Me ',                                                                        
                                }}, regex=True)

df= df.replace({'ICI ART Number': { 'ICI ART[/$0-9,A-Za-z .>]*2468102246810Add to Cart$': ' EvenMax10 ', 
                                    'ICI ART[/$0-9,A-Za-z .>]*246822468Add to Cart$': ' EvenMax8 ',
                                    'ICI ART[/$0-9,A-Za-z .>]*2462246Add to Cart$': ' EvenMax6 ',
                                    'ICI ART[/$0-9,A-Za-z .>]*24224Add to Cart$': ' EvenMax4 ',
                                    'ICI ART[/$0-9,A-Za-z .>]*Please Call$': ' Please Call ',   
                                    'ICI ART[/$0-9,A-Za-z .>]*Notify Me$': ' Notify Me ',                                                                        
                                }}, regex=True)
df= df.replace({'IVF Number': { 'IVFIn[/$0-9,A-Za-z .>]*12345678910112345678910Add to Cart$': ' Max10 ', 
                                'IVFIn[/$0-9,A-Za-z .>]*1234567891123456789Add to Cart$': ' Max9 ',
                                'IVFIn[/$0-9,A-Za-z .>]*12345678112345678Add to Cart$': ' Max8 ',
                                'IVFIn[/$0-9,A-Za-z .>]*123456711234567Add to Cart$': ' Max7 ',
                                'IVFIn[/$0-9,A-Za-z .>]*1234561123456Add to Cart$': ' Max6 ',
                                'IVFIn[/$0-9,A-Za-z .>]*12345112345Add to Cart$': ' Max5 ',
                                'IVFIn[/$0-9,A-Za-z .>]*Please Call$': ' Please Call ',   
                                'IVFIn[/$0-9,A-Za-z .>]*Notify Me$': ' Notify Me ',                                                                        
                                }}, regex=True)


df['IUI Price']= df['IUI1']
df['IUI ART Price']= df['IUI ART']
df['ICI Price']= df['ICI']
df['ICI ART Price']= df['ICI ART']
df['IVF Price']= df['IVF']

df['IUI Price']= df['IUI1'].str.extract(pat='(\$[0-9,]{1,5})')
df['IUI ART Price']= df['IUI ART'].str.extract(pat='(\$[0-9,]{1,5})')
df['ICI Price']= df['ICI'].str.extract(pat='(\$[0-9,]{1,5})')
df['ICI ART Price']= df['ICI ART'].str.extract(pat='(\$[0-9,]{1,5})')
df['IVF Price']= df['IVF'].str.extract(pat='(\$[0-9,]{1,5})')


#counts = df['IUI ART Number'].explode().value_counts()
#print(counts)
#df.to_csv('df.csv') 
#files.download('df.csv')


In [ ]:
df.to_csv('%s.csv'%date1) 
files.download('%s.csv'%date1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#uploaded = files.upload()